In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('../combined.csv')
# get rid of unamed column
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,document,clause,severity,a,ch,cr,j,law,ltd,ter,use
0,Spotify,Spotify,0,0,0,0,0,0,0,0,0
1,Spotify,•\tPremium,0,0,0,0,0,0,0,0,0
2,Spotify,•\tHelp,0,0,0,0,0,0,0,0,0
3,Spotify,•\tDownload,0,0,0,0,0,0,0,0,0
4,Spotify,•,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
12312,Vivino,Where Vivino has provided you with a translati...,0,0,0,0,0,0,0,0,0
12313,Vivino,If there is any contradiction between what the...,0,0,0,0,0,0,0,0,0
12314,Vivino,Contact,0,0,0,0,0,0,0,0,0
12315,Vivino,You may contact Vivino at the following addres...,0,0,0,0,0,0,0,0,0


In [20]:
data_cleaned = df
X = data_cleaned['clause']
y_tags = data_cleaned[['a', 'ch', 'cr', 'j', 'law', 'ltd', 'ter', 'use']]
y_severity = data_cleaned['severity']


In [45]:
# model to predict severity of the clause
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier


# Splitting the data into training and testing sets for severity
X_train_sev, X_test_sev, y_train_sev, y_test_sev = train_test_split(X, y_severity, test_size=0.2, random_state=42)

# Text Preprocessing: Reusing the earlier vectorizer
X_train_sev_tfidf = vectorizer.transform(X_train_sev)
X_test_sev_tfidf = vectorizer.transform(X_test_sev)

# Model for Severity: Random Forest Classifier
severity_classifier = RandomForestClassifier(n_estimators=100)

# Training the model for severity
severity_classifier.fit(X_train_sev_tfidf, y_train_sev)

# Predicting on the test set for severity
y_pred_sev = severity_classifier.predict(X_test_sev_tfidf)

# Evaluation for Severity
accuracy_sev = accuracy_score(y_test_sev, y_pred_sev)

#  cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(severity_classifier, X_train_sev_tfidf, y_train_sev, cv=5)



/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


0.9389019977490876
0.9403409090909091
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 2
0 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
2 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 2
0 0
0 0
0 0
0 0
0 0
0 2
0 0
0 0
0 2
0 0
0 0
0 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 2
0 0
0 0
0 0
0 0
0 0
0 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
2 2
0 0
0 0
0 0
0 0
0 0
2 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 3
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 2
0 0
0 0
2 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
2 3
0 0
0 0
0 2
0 0
0 0
0 0
0 0
0 0
2 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
2 2
0 0
0 0
0 0
0 0
0 0
0 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 3
0 0
0 

In [54]:
# create data frame for y_pred_sev with column name 'severity'
y_pred_sev_df = pd.DataFrame(y_pred_sev, columns=['severity'])
y_pred_sev_df['severity'].value_counts()

severity
0    2379
2      61
3      21
1       3
Name: count, dtype: int64

In [48]:
import pandas as pd
new_data = pd.read_csv('../data/tosdr_data.csv')
new_clauses = new_data['title']

# Predicting on the new data
new_clauses_tfidf = vectorizer.transform(new_clauses)
new_pred_sev = severity_classifier.predict(new_clauses_tfidf)


# create new dataframe with new clauses and predicted severity
new_df = pd.DataFrame({'clause': new_clauses, 'severity': new_pred_sev})
new_df

,clause,severity
0,You can delete your content from this service,0
1,A license is kept on user-generated content ev...,0
2,This service tracks you on other websites,0
3,You agree to comply with the law of the servic...,0
4,This service retains rights to your content ev...,0
...,...,...
248,"You agree to defend, indemnify, and hold the s...",0
249,This service assumes no liability for any loss...,0
250,Information is provided about what kind of inf...,0
251,Your personal data is used for automated decis...,0


In [55]:
new_df['severity'].value_counts()

severity
0    251
2      2
Name: count, dtype: int64